# Start of the Project

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163760 sha256=7eb10127e4c6862e56d4d43e0e8285e2f7bc74b4b53059d91a89fc3856e5c82f
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import gc
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.sparse.linalg import svds
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from scipy.sparse import csr_matrix
from surprise import KNNBasic
from surprise import accuracy
from surprise import SVD
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)

In [ ]:
CustomerData = pd.read_csv('/content/drive/MyDrive/UnSupervisedMachineLearningData/customers.csv',sep=',')

In [ ]:
ArticlesData = pd.read_csv('/content/drive/MyDrive/UnSupervisedMachineLearningData/articles.csv',sep=',', dtype={'article_id': str})

In [ ]:
TransactionData = pd.read_csv('/content/drive/MyDrive/UnSupervisedMachineLearningData/transactions_train.csv',sep=',', dtype={'article_id': str} , parse_dates=['t_dat'])

In [ ]:
TransactionData = TransactionData.drop(['price','sales_channel_id'],axis = 1)

In [ ]:
TransactionData = TransactionData[TransactionData['t_dat'] >= "2020-09-01"]

In [ ]:
TransactionData = TransactionData.drop('t_dat',axis=1)

In [ ]:
TransactionData = TransactionData.reset_index(drop=True)

In [ ]:
TransactionData = TransactionData.groupby(['customer_id','article_id']).size().reset_index(name='Count')

In [ ]:
TransactionData

,customer_id,article_id,Count
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043,1
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007,1
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0448509014,1
3,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0719530003,1
4,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,0685814001,3
...,...,...,...
705499,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0804992033,1
705500,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0689365050,1
705501,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0762846027,1
705502,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0794819001,1


In [ ]:
len(TransactionData['customer_id'].unique())

189510

In [ ]:
len(TransactionData['article_id'].unique())

26252

In [ ]:
customer_index = TransactionData['customer_id'].unique()
product_index = TransactionData['article_id'].unique()
sparse_matrix = csr_matrix((TransactionData['Count'],
                           (pd.Categorical(TransactionData['customer_id'], categories=customer_index).codes,
                            pd.Categorical(TransactionData['article_id'], categories=product_index).codes)))


In [ ]:
print(sparse_matrix)

  (0, 0)	1
  (1, 1)	1
  (2, 2)	1
  (2, 3)	1
  (3, 4)	3
  (4, 5)	1
  (4, 6)	1
  (4, 7)	1
  (4, 8)	1
  (4, 9)	1
  (4, 10)	1
  (4, 11)	1
  (4, 12)	1
  (4, 13)	1
  (4, 14)	1
  (5, 15)	1
  (6, 16)	1
  (7, 17)	1
  (8, 18)	1
  (8, 19)	1
  (8, 20)	1
  (8, 21)	2
  (8, 22)	1
  (8, 23)	1
  (8, 24)	1
  :	:
  (189505, 902)	1
  (189505, 1304)	1
  (189505, 2372)	1
  (189505, 3994)	1
  (189505, 4216)	1
  (189506, 174)	1
  (189506, 1460)	1
  (189507, 254)	1
  (189507, 318)	2
  (189507, 521)	1
  (189507, 1270)	1
  (189507, 1940)	1
  (189507, 2007)	1
  (189507, 3159)	2
  (189507, 4198)	2
  (189508, 316)	1
  (189508, 715)	1
  (189508, 2345)	1
  (189508, 2977)	1
  (189508, 5414)	1
  (189508, 10900)	1
  (189509, 894)	1
  (189509, 6196)	1
  (189509, 8611)	1
  (189509, 12326)	1


In [ ]:
reader = Reader(line_format='user item rating', rating_scale=(sparse_matrix.min(), sparse_matrix.max()))
data = Dataset.load_from_df(TransactionData[['customer_id', 'article_id', 'Count']], reader)

In [ ]:
data

In [ ]:
algo = SVD()
algo.fit(data.build_full_trainset())

In [ ]:
TransactionData['customer_id'].unique()[:25]

array(['00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
       '000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318',
       '0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d8cd0c725276a467a2a',
       '000172a9c322560c849754ffbdfdb2180d408aa7176b943f957804686be8e1f0',
       '0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94',
       '0001f8cef6b9702d54abf66fd89eb21014bf98567065a9b5e42f37bc99528cf5',
       '0002cca4cc68601e894ab62839428e5f0696417fe0f9e84551c6827a7629d441',
       '00039306476aaf41a07fed942884f16b30abfa83a2a8bea972019098d6406793',
       '0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf4672f30b3e622fec55',
       '00040239317e877c77ac6e79df42eb2633ad38fcac09fc0094e549180ddc201c',
       '000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed6396773839f6bf71a9',
       '000525e3fe01600d717da8423643a8303390a055c578ed8a97256600baf54565',
       '0005340aa69bb5a28d98712a36d8f669024bce137e3c82a618ea49032b9023c6',
       '0005ed68483efa396

Building Recommendation Funciton

In [ ]:
user_id = '0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf4672f30b3e622fec55'

In [ ]:
def Top10Recommendations(user_id):
  items_bought_by_customer = TransactionData[TransactionData['customer_id'] == user_id]['article_id'].tolist()
  items_not_bought_by_customer = [item for item in TransactionData['article_id'].unique() if item not in items_bought_by_customer]
  predictions_for_user = [algo.predict(user_id, item) for item in items_not_bought_by_customer]
  top_n_recommendations = sorted(predictions_for_user, key=lambda x: x.est, reverse=True)[:10]
  recommendations = []
  for recommendation in top_n_recommendations:
    recommendations.append(recommendation.iid)
  return recommendations


In [ ]:
Top10Recommendations("0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94")

['0715255013',
 '0717464001',
 '0846407001',
 '0877607001',
 '0685814001',
 '0918443002',
 '0820866001',
 '0541758001',
 '0395127011',
 '0201219001']